# Y plus wall distance estimation

Normally working with two types of **Turbulent models**:
 - $k-\epsilon$ model
 - $k-\omega$ models: *kOmegaSST*, *kOmegaSSTLM*, *kOmegaSSTSAS*

For details look at [https://www.openfoam.com/documentation/guides/latest/doc/guide-turbulence.html](OpenFOAM turbulence guide).

## Boundary layer description

Effective use of turbulence models requires close attention to their respective meshing requirements, particularly in near-wall regions. In the figure below it is seen the flow development in the near wall region

    <img src="figs/turbulence-wall-profile.png">

Different RAS models requires different y+ valuesto define the size of first layer thickness ():
 - $k-\epsilon$ type models with **low Re** models requires **y+ < 0.2**, **high Re** requires **10 < y+ < 100**
 - $k-\omega$ type models:
   - **low Re**:first cell height **y+ < 2** (10 to 20 layers with scaling factor 1.1),
   - **high Re**: first cell height requires **30 < y+ < 200** (*kOmegaSST*, number of cells is calculated by the BL thicknes and scaling factor)
 - new $k-\omega$ type models (*kOmegaSSTTLM*, *kOmegaSSTSAS*) y+ formulation requires **y+ < 10** for the first layer thickness,
 - transitional flow requires **y+ <1** + very good stream wise mesh distribution (when you expect boundary layer deatacments),
 - in addition, also needed **alteast 15-40** layers inside boundary layer besides the y+ requirements,
 - boundary mesh inflation scailing factor should be **1.2 < sf < 1.5**.


When meshing it is often useful to be able to estimate the wall-distance needed to obtain a certain Y+ value. To estimate this you can do the following:


1. Compute the Re number:

$$
Re_x = \frac{\rho \cdot U_\text{freestream} \cdot L_\text{boundary\,layer}}{\mu}
$$

$L=x$ it is a length of boundary layer (foilr cord length).

2. Estimate the skin friction using one of the formulas given here, for example, using the Schlichting skin-friction correlation:

$$
C_f = [ 2 \, \log_{10}(Re_x) - 0.65 ] ^{-2.3} \quad \mbox{for} \quad Re_x < 10^9 
$$

3. Compute the Wall shear stress:

$$
\tau_w = C_f \cdot \frac{1}{2} \, \rho \, U_\text{freestream}^2
$$

4. Compute the Friction velocity:

$$
u_{\tau} = \sqrt{\frac{\tau_w}{\rho}}
$$

5. Compute the wall distance:

$$
y = \frac{y^+ \mu}{\rho \, u_{\tau}}
$$

6. Number of layers can be estimated by the boundary layer thickness $\delta$:

$$
    \delta = x \:0.37 \: Re_x^{-1/5}
$$

In [1]:
import math as mat

In [2]:
def wall_distance(rho, mu, U, L, yPlus):

    Re = rho * U * L / mu
    Cf = (2 * mat.log10(Re) - 0.65)**(-2.3)
    tau_w = Cf * 0.5 * rho * U**2
    uf = mat.sqrt(tau_w/rho)

    y = (yPlus * mu) / (rho * uf)

    return [y, Re]

In [3]:
def number_of_layers(h1, sf, Re, L):

    # boundary layer thickness
    d = L * 0.37 * Re**(-1/5) 

    # estimate number of layers to cover BL delta
    # from equation H = h1 * (1 - sf**(N-1))/(1 - sf), describing total height of N layers with inflation scaling factor sf,
    # we derive N setting d = H
    N = mat.ceil(1 + mat.log(1 - d/h1*(1-sf))/mat.log(sf))

    return [d, N]

In [4]:
def mesh_calculator(h1, sf, N):

    sr = sf**N   # OF scale factor hn/h1
    hn = h1 * sr # height of the last Nth layer
    H = h1 * (1 - sr)/(1 - sf) # total height of N layers

    return [H, hn, sr]

In [7]:
def get_OF_grading(bl_H, bl_sr, bl_hN, bl_N, H2, hN2, N2):

    p_bl = bl_H/(bl_H + H2) * 100  # percent of total length
    sf_2 = hN2/bl_hN  # OF grading factor for zone 2

    str =  '(\n'
    str += '\t({:.1f} {:d} {:.2f})\n'.format(p_bl, bl_N, bl_sr)
    str += '\t({:.1f} {:d} {:.2f})\n'.format(100-p_bl, N2, sf_2)
    str += ')\n'

    return str

In [13]:
# *** Main ***

# first cell BL thickness
rho = 1000 # density [kg/m3]
mu = 1e-3 # dynamic viscosity [Pa s]
U = 5 # freestream velocity [m/s]
L = 1 # length scale in flow [m]; length of surface
yP = 10 # desidered y+

[y, Re] = wall_distance(rho, mu, U, L, yP)

# mesh construction factors
bl_sf = 1.2 # inflation scaling factor
# number of layers
[delta, bl_N] = number_of_layers(y, bl_sf, Re, L)

print('Near wall distace parameters')
print(' -> Re = {:.3e} (U={:.3e} m/s, L={:.3e} m, nu={:.3e} m2/s)'.format(Re,U,L,rho/mu))
print(' -> Y+ = {:.2f} (lower limit y+)'.format(yP))
print(' -> h1 = {:.3e} m  (first near wall BL cell height)'.format(y))
print(' ->  d = {:.3e} m  (calculated BL thickness)'.format(delta))

[bl_H, bl_hN, bl_sr] = mesh_calculator(y, bl_sf, bl_N)
print()
print('Bundary layer mesh parameters:')
print(' ->  bl_sf = {:.2f} m  (BL scaling factor)'.format(bl_sf))
print(' ->   bl_N = {:d} m  (number of BL layers)'.format(bl_N))
print(' ->   bl_H = {:.3e} m  (total BL height)'.format(bl_H))
print(' ->  bl_hN = {:.3e} m  (last BL layer height)'.format(bl_hN))
print(' ->  bl_sr = {:.1f}  (grading factor for OF blockMesh dict)'.format(bl_sr))

# OF grading string
H2 = 3*bl_H    # lenght of the cell edge in BL direction
N2 = 2*bl_N    # number of total divisions
hN2 = 10*bl_hN # size of tha final cell, near new block

print()
print('Above bundary layer zone mesh parameters:')
print(' ->  H2 = {:.3e} m  (total thickness of second zone)'.format(H2))
print(' -> hN2 = {:.3e} m  (last layer thickness)'.format(hN2))
print(' ->  N2 = {:.1f}  (number of layer in second zone)'.format(N2))

of_gstr = get_OF_grading(bl_H, bl_sr, bl_hN, bl_N, H2, hN2, N2)
print()
print('OpenFOAM cell edge grading scenario for blockMesh')
print(of_gstr)

Near wall distace parameters
 -> Re = 5.000e+06 (U=5.000e+00 m/s, L=1.000e+00 m, nu=1.000e+06 m2/s)
 -> Y+ = 10.00 (lower limit y+)
 -> h1 = 5.282e-05 m  (first near wall BL cell height)
 ->  d = 1.692e-02 m  (calculated BL thickness)

Bundary layer mesh parameters:
 ->  bl_sf = 1.20 m  (BL scaling factor)
 ->   bl_N = 24 m  (number of BL layers)
 ->   bl_H = 2.073e-02 m  (total BL height)
 ->  bl_hN = 4.199e-03 m  (last BL layer height)
 ->  bl_sr = 79.5  (grading factor for OF blockMesh dict)

Above bundary layer zone mesh parameters:
 ->  H2 = 6.219e-02 m  (total thickness of second zone)
 -> hN2 = 4.199e-02 m  (last layer thickness)
 ->  N2 = 48.0  (number of layer in second zone)

OpenFOAM cell edge grading scenario for blockMesh
(
	(25.0 24 79.50)
	(75.0 48 10.00)
)

